In [1]:
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize, dual_annealing
import matplotlib.pyplot as plt
import concurrent.futures
import threading, multiprocessing

import sys 
sys.path.append("../")  
import minionpy as mpy
import minionpy.test_functions as mpytest
import time
import iminuit

# Minimizing CEC benchmark problems with noise


In [2]:
# Global variables
N = 0  # Global counter for function evaluations
noise_ratio = 1e-4  # Noise level for the objective function
N_dict = {}  # Stores the number of function evaluations per algorithm
algos = ["ARRDE", "NelderMead", "L_BFGS_B", "DA"] #here, DA is dual annealing, ABC=artificial bee colony

def test_optimization_noise(func_number, year, bounds, dimension, func_name, Nmaxeval, seed):
    """
    Runs multiple optimization algorithms on the given function and stores results.
    
    Parameters:
    - func: Objective function to be minimized
    - bounds: Tuple representing the search space bounds
    - dimension: Number of dimensions for the problem
    - func_name: Name of the function (used for logging results)
    - Nmaxeval: Maximum number of function evaluations
    - seed: Random seed for reproducibility
    """
    global results, N, N_dict, noise_ratio, results_lock, algos

    if year == 2014:
        cec_func = mpy.CEC2014Functions(function_number=func_number, dimension=dimension)
    elif year == 2017:
        cec_func = mpy.CEC2017Functions(function_number=func_number, dimension=dimension)
    elif year == 2019:
        cec_func = mpy.CEC2019Functions(function_number=func_number)
    elif year == 2020:
        cec_func = mpy.CEC2020Functions(function_number=func_number, dimension=dimension)
    elif year == 2022:
        cec_func = mpy.CEC2022Functions(function_number=func_number, dimension=dimension)
    else:
        raise Exception("Unknown CEC year.")
    
    func = cec_func
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    bounds_list = [bounds] * dimension  # Extend bounds to all dimensions
    x0 = [0.0 for _ in range(dimension)]  # Initial starting point

    def func_wrapper(X):
        """Wraps the function to add evaluation tracking and noise."""
        global N
        ret = np.array(func(X))  # Compute function value
        N += len(X)  # Track the number of function evaluations
        return ret + noise_ratio * np.random.normal(size=len(X)) * np.abs(ret)  # Add noise
    
    def func_scipy(par):
        """Wrapper for compatibility with SciPy optimization methods."""
        return func_wrapper([par])[0]

    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 1,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=1"] = res.fun
    N_dict["L_BFGS_B N=1"] = N

    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 3,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=3"] = res.fun
    N_dict["L_BFGS_B N=3"] = N


    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 5,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=5"] = res.fun
    N_dict["L_BFGS_B N=5"] = N


    N = 0  # Reset evaluation counter
    res = mpy.Minimizer(
        func_wrapper, bounds_list, x0=x0, relTol=0.0, algo="L_BFGS_B", maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 7,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS_B N=7"] = res.fun
    N_dict["L_BFGS_B N=7"] = N

    
    # Run L-BFGS-B from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0, method="L-BFGS-B", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Scipy L_BFGS_B"] = res_minimize.fun
    N_dict["Scipy L_BFGS_B"] = N

    # Run Minuit Migrad from iminuit
    N = 0
    res_minimize = iminuit.minimize(func_scipy, x0=x0, method="migrad", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Minuit Migrad"] = res_minimize.fun
    N_dict["Minuit Migrad"] = N
    
    # Print results
    for res in result:
        if res == "Function" : 
            print("Function : ", result[res])
        if res not in ['Dimensions', 'Function']:
            print(f"\t{res:<20} : {result[res]:<10} \t N_evals : {N_dict[res]:<10}")
    print("")

def test_optimization(func, bounds, dimension, func_name, Nmaxeval, seed):
    """
    Runs multiple optimization algorithms on the given function and stores results.
    
    Parameters:
    - func: Objective function to be minimized
    - bounds: Tuple representing the search space bounds
    - dimension: Number of dimensions for the problem
    - func_name: Name of the function (used for logging results)
    - Nmaxeval: Maximum number of function evaluations
    - seed: Random seed for reproducibility
    """
    global results, N, N_dict, noise_ratio, results_lock, algos
    
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    bounds_list = [bounds] * dimension  # Extend bounds to all dimensions
    x0 = [0.0 for _ in range(dimension)]  # Initial starting point

    def func_wrapper(X):
        """Wraps the function to add evaluation tracking and noise."""
        global N
        ret = np.array(func(X))  # Compute function value
        N += len(X)  # Track the number of function evaluations
        return ret + noise_ratio * np.random.normal(size=len(X)) * np.abs(ret)  # Add noise
    
    def func_scipy(par):
        """Wrapper for compatibility with SciPy optimization methods."""
        return func_wrapper([par])[0]
    
    # Run various optimization algorithms
    for algo in algos:
        N = 0  # Reset evaluation counter
        res = mpy.Minimizer(
            func_wrapper, bounds_list, x0=x0, relTol=0.0, algo=algo, maxevals=Nmaxeval, 
            callback=None, seed=seed, options={
                "population_size": 0, 
                "N_points_derivative": 3,
                "use_local_search": True,
                "func_noise_ratio": noise_ratio
            }
        ).optimize()
        result[algo] = res.fun
        N_dict[algo] = N

    # Run L-BFGS method from minionpy
    N = 0
    res = mpy.L_BFGS(
        func_wrapper, x0=x0, relTol=0.0, maxevals=Nmaxeval, 
        callback=None, seed=seed, options={
            "population_size": 0, 
            "N_points_derivative": 3,
            "use_local_search": True,
            "func_noise_ratio": noise_ratio
        }
    ).optimize()
    result["L_BFGS"] = res.fun
    N_dict["L_BFGS"] = N
    
    # Run L-BFGS-B from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0, method="L-BFGS-B", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Scipy L_BFGS_B"] = res_minimize.fun
    N_dict["Scipy L_BFGS_B"] = N
    
    # Run Dual Annealing from SciPy
    N = 0
    dual_ann = dual_annealing(func_scipy, bounds_list, maxfun=Nmaxeval, no_local_search=False, x0=x0)
    result["Scipy DA"] = dual_ann.fun
    N_dict["Scipy DA"] = N

    # Run Nelder-Mead from SciPy
    N = 0
    res_minimize = minimize(func_scipy, x0=x0, method="Nelder-Mead", options={"maxfev": Nmaxeval, "adaptive": True}, bounds=bounds_list)
    result["Scipy NelderMead"] = res_minimize.fun
    N_dict["Scipy NelderMead"] = N

    # Run Minuit Migrad from iminuit
    N = 0
    res_minimize = iminuit.minimize(func_scipy, x0=x0, method="migrad", options={"maxfun": Nmaxeval}, bounds=bounds_list)
    result["Minuit Migrad"] = res_minimize.fun
    N_dict["Minuit Migrad"] = N
    
    # Print results
    for res in result:
        if res == "Function" : 
            print("Function : ", result[res])
        if res not in ['Dimensions', 'Function']:
            print(f"\t{res:<20} : {result[res]:<10} \t N_evals : {N_dict[res]:<10}")
    print("")

def run_test_optimization(j, dim, year=2017, seed=None, Nmaxeval=10000):
    """
    Runs optimization tests for a specified CEC benchmark function.
    
    Parameters:
    - j: Function index in the CEC benchmark set
    - dim: Dimensionality of the function
    - year: Year of the CEC benchmark suite (default: 2017)
    - seed: Random seed for reproducibility
    - Nmaxeval: Maximum number of function evaluations
    """
    if year == 2014:
        cec_func = mpy.CEC2014Functions(function_number=j, dimension=dim)
    elif year == 2017:
        cec_func = mpy.CEC2017Functions(function_number=j, dimension=dim)
    elif year == 2019:
        cec_func = mpy.CEC2019Functions(function_number=j)
    elif year == 2020:
        cec_func = mpy.CEC2020Functions(function_number=j, dimension=dim)
    elif year == 2022:
        cec_func = mpy.CEC2022Functions(function_number=j, dimension=dim)
    else:
        raise Exception("Unknown CEC year.")
    
    test_optimization(cec_func, (-100, 100), dim, "func_" + str(j), Nmaxeval, seed)


# Performance of Minion's L-BFGS-B with Different $N$ Derivative Points

Minion's L-BFGS-B is vectorized and designed to be robust against noise. Function evaluations and their derivatives are computed in batches, ensuring efficient execution. To estimate derivatives, Minion employs the noise-robust Lanczos derivative method.

In the L-BFGS-B and L-BFGS settings, the key parameter `'N_points_derivative'` determines the number of points used for derivative calculation. This notebook compares the performance of L-BFGS-B with different values of `'N_points_derivative'`. A noise level of $10^{-4}$ is added to the CEC2017 benchmark problems to simulate real-world conditions.

- When $N = 1$, the numerical derivative reduces to the standard forward difference method.
- For $N \geq 2$, the Lanczos derivative formula is used.
- Specifically, $N = 3$ corresponds to the central difference method.

The following sections analyze how different values of `'N_points_derivative'` impact optimization performance.

In [3]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-4

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection

year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

for j in func_numbers:
    test_optimization_noise(j, year, (-100, 100), dimension, "func_"+str(j), Nmaxeval, None)

Function :  func_1
	L_BFGS_B N=1         : 463248.4729853859 	 N_evals : 1452      
	L_BFGS_B N=3         : 12734.836113542968 	 N_evals : 6846      
	L_BFGS_B N=5         : 1439.90115969665 	 N_evals : 6478      
	L_BFGS_B N=7         : 101.27112567415122 	 N_evals : 9333      
	Scipy L_BFGS_B       : 29977690348.50332 	 N_evals : 231       
	Minuit Migrad        : 29971509326.080196 	 N_evals : 2780      

Function :  func_2
	L_BFGS_B N=1         : 76711.56342742995 	 N_evals : 2530      
	L_BFGS_B N=3         : 278.0263258817032 	 N_evals : 5124      
	L_BFGS_B N=5         : 343.29417405433577 	 N_evals : 8487      
	L_BFGS_B N=7         : 63241.73613264353 	 N_evals : 16226     
	Scipy L_BFGS_B       : 8.870637956725052e+17 	 N_evals : 550       
	Minuit Migrad        : 8.868297170716698e+17 	 N_evals : 2580      

Function :  func_3
	L_BFGS_B N=1         : 19784.375983096343 	 N_evals : 1793      
	L_BFGS_B N=3         : 300.48644719351597 	 N_evals : 5271      
	L_BFGS_B N=5     

We observe that with $N=1$, where $N$ is the number of points used in the derivative calculation, L-BFGS-B is more robust than both SciPy's L-BFGS-B and Minuit Migrad. Notably, Minuit Migrad is a variant of the BFGS algorithm that has been widely used in high-energy physics for over 40 years due to its robustness. Our results confirm that Minuit Migrad is generally more robust than SciPy's L-BFGS-B.

When using higher values of $N$, we see that $N=3$ generally improves robustness but also requires more function evaluations. Increasing $N$ further, such as to $N=7$, does not provide significant improvements compared to using $N=3$ or $N=5$. Based on these findings, we recommend using either $N=3$ or $N=5$ as a trade-off between robustness and computational cost.

The number of function evaluations required for computing the function and its derivative, given $N$, is given by:

$$
\text{Function Calls} = 1 + D \cdot (N - 1)
$$

where $D$ is the dimensionality of the problem. These function evaluations are performed in batches to enhance efficiency.

What Happens if the Function is Smooth?
To analyze performance in a noise-free setting, we can compare results by setting the noise level to zero.

In [4]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 0.0

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection

year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

for j in func_numbers:
    test_optimization_noise(j, year, (-100, 100), dimension, "func_"+str(j), Nmaxeval, None)

Function :  func_1
	L_BFGS_B N=1         : 100.02404359945164 	 N_evals : 1144      
	L_BFGS_B N=3         : 100.0      	 N_evals : 903       
	L_BFGS_B N=5         : 100.0      	 N_evals : 1804      
	L_BFGS_B N=7         : 100.0      	 N_evals : 2623      
	Scipy L_BFGS_B       : 100.00000382406891 	 N_evals : 330       
	Minuit Migrad        : 100.00000102982862 	 N_evals : 802       

Function :  func_2
	L_BFGS_B N=1         : 200.0000732365787 	 N_evals : 2574      
	L_BFGS_B N=3         : 200.00329242873912 	 N_evals : 4053      
	L_BFGS_B N=5         : 200.00001167305413 	 N_evals : 8938      
	L_BFGS_B N=7         : 200.00048981628413 	 N_evals : 14701     
	Scipy L_BFGS_B       : 200.0002490787248 	 N_evals : 1441      
	Minuit Migrad        : 200.00010737120294 	 N_evals : 3479      

Function :  func_3
	L_BFGS_B N=1         : 300.00000000005656 	 N_evals : 627       
	L_BFGS_B N=3         : 300.0000000000012 	 N_evals : 1197      
	L_BFGS_B N=5         : 300.0000000000004 	 

We can see that using higher $N$ does not improves the performance. Therefore, for smooth function, $N=1$ can be safely used. 

# Performance of Minion's L-BFGS-B at Different Noise Levels

In this section, we compare the performance of various algorithms implemented in Minion against their counterparts in other libraries, such as SciPy and Minuit. The test function used is the CEC2017 benchmark function with a dimensionality of $D = 10$.

## Noise Level : 0.01

In [5]:
# Counter for function evaluations
N = 0

# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 0.01

# Dictionary to store the number of evaluations per algorithm
N_dict = {}

# Maximum number of function evaluations allowed per optimization run
Nmaxeval = 100000  

# Dimensionality of the optimization problem
dimension = 10 

# Number of times each function should be tested (repetitions)
NRuns = 1  

# The CEC benchmark year to use for function selection
year = 2017  

# Dictionary mapping CEC benchmark years to their respective function sets
func_numbers_dict = {
    2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 
    2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
}

# Retrieve the function numbers for the selected benchmark year
func_numbers = func_numbers_dict[year]

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 96.83123987305935 	 N_evals : 100044    
	NelderMead           : 430908.50804423104 	 N_evals : 100004    
	L_BFGS_B             : 28941438556.54684 	 N_evals : 1995      
	DA                   : 4588.874587578041 	 N_evals : 100011    
	L_BFGS               : 29171976960.169834 	 N_evals : 1491      
	Scipy L_BFGS_B       : 30283549235.90995 	 N_evals : 275       
	Scipy DA             : 128.2698412131467 	 N_evals : 43695     
	Scipy NelderMead     : 28769662418.353573 	 N_evals : 100000    
	Minuit Migrad        : 29398999596.040783 	 N_evals : 1992      

Function :  func_2
	ARRDE                : 191.7761833802127 	 N_evals : 100045    
	NelderMead           : 157421.01356714347 	 N_evals : 100008    
	L_BFGS_B             : 485986386.6784201 	 N_evals : 9660      
	DA                   : 223.65401453574555 	 N_evals : 100004    
	L_BFGS               : 8.15451278272593e+17 	 N_evals : 1365      
	Scipy L_BFGS_B       : 8.899142694749993e

## Noise Level: 0.0001 

In [6]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-4

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.97213672285594 	 N_evals : 100044    
	NelderMead           : 238896345.3423589 	 N_evals : 100009    
	L_BFGS_B             : 4202.355898311541 	 N_evals : 4032      
	DA                   : 2696.954654389883 	 N_evals : 100003    
	L_BFGS               : 29962386702.831707 	 N_evals : 2604      
	Scipy L_BFGS_B       : 29976695381.37704 	 N_evals : 319       
	Scipy DA             : 2718.1855044554127 	 N_evals : 48227     
	Scipy NelderMead     : 29961935863.871994 	 N_evals : 100000    
	Minuit Migrad        : 29972164033.977577 	 N_evals : 3055      

Function :  func_2
	ARRDE                : 199.92667579112364 	 N_evals : 100045    
	NelderMead           : 42689.32373770773 	 N_evals : 100004    
	L_BFGS_B             : 318.6091227624048 	 N_evals : 4914      
	DA                   : 199.93487554097686 	 N_evals : 100008    
	L_BFGS               : 7.574900447268609e+17 	 N_evals : 1365      
	Scipy L_BFGS_B       : 8.869009728759104

## Noise Level : 1e-6

In [7]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-6

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(1):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.9997343824917 	 N_evals : 100045    
	NelderMead           : 206692916.4742705 	 N_evals : 100006    
	L_BFGS_B             : 99.99976894890757 	 N_evals : 4893      
	DA                   : 99.99966750653815 	 N_evals : 100016    
	L_BFGS               : 100.84196305282168 	 N_evals : 2121      
	Scipy L_BFGS_B       : 29975463562.683376 	 N_evals : 231       
	Scipy DA             : 1189.763459587311 	 N_evals : 64782     
	Scipy NelderMead     : 3506009578.4551578 	 N_evals : 100000    
	Minuit Migrad        : 100.00379479018459 	 N_evals : 6509      

Function :  func_2
	ARRDE                : 199.99938699767105 	 N_evals : 100044    
	NelderMead           : 42484.311927869385 	 N_evals : 100007    
	L_BFGS_B             : 200.00028413329144 	 N_evals : 6048      
	DA                   : 200.00057312282777 	 N_evals : 100014    
	L_BFGS               : 7.556722288675401e+17 	 N_evals : 1365      
	Scipy L_BFGS_B       : 8.86964137038380

## Noise Level : 1e-8

In [8]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-8

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.99999660720574 	 N_evals : 100044    
	NelderMead           : 215885579.7319631 	 N_evals : 100008    
	L_BFGS_B             : 99.99999720627525 	 N_evals : 2688      
	DA                   : 99.9999962431606 	 N_evals : 100021    
	L_BFGS               : 99.99999799871546 	 N_evals : 1974      
	Scipy L_BFGS_B       : 29975432225.473763 	 N_evals : 319       
	Scipy DA             : 387.01950359026904 	 N_evals : 32178     
	Scipy NelderMead     : 99.99999831582275 	 N_evals : 9779      
	Minuit Migrad        : 100.00000603780694 	 N_evals : 841       

Function :  func_2
	ARRDE                : 199.99999254443566 	 N_evals : 100044    
	NelderMead           : 42484.22873638728 	 N_evals : 100000    
	L_BFGS_B             : 200.00248460809996 	 N_evals : 4935      
	DA                   : 200.0000861635431 	 N_evals : 100014    
	L_BFGS               : 7.556707089130984e+17 	 N_evals : 1365      
	Scipy L_BFGS_B       : 128589184183714.12 

## Noise Level : 1e-10

In [9]:
# Noise ratio for function evaluations (set to zero for noiseless optimization)
noise_ratio = 1e-10

# Using a thread pool to execute optimization tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = []  # List to store future objects representing scheduled tasks

    # Run optimization tests multiple times (for averaging results)
    for k in range(NRuns):
        for j in func_numbers:
            # Submit the optimization test function to the thread pool
            futures.append(executor.submit(run_test_optimization, j, dimension, year, k, Nmaxeval))

    # Wait for all submitted tasks to complete
    concurrent.futures.wait(futures)

    # Retrieve and process results (ensure all threads completed successfully)
    for f in futures:
        f.result()

Function :  func_1
	ARRDE                : 99.99999997148959 	 N_evals : 100045    
	NelderMead           : 215885579.1742297 	 N_evals : 100007    
	L_BFGS_B             : 99.99999999049744 	 N_evals : 987       
	DA                   : 99.99999996591855 	 N_evals : 100011    
	L_BFGS               : 99.99999998670438 	 N_evals : 861       
	Scipy L_BFGS_B       : 27689573623.689964 	 N_evals : 275       
	Scipy DA             : 100.02003483102352 	 N_evals : 21816     
	Scipy NelderMead     : 2712.3754026169663 	 N_evals : 6914      
	Minuit Migrad        : 100.00000021002626 	 N_evals : 813       

Function :  func_2
	ARRDE                : 199.99999994768564 	 N_evals : 100044    
	NelderMead           : 42484.22907962483 	 N_evals : 100003    
	L_BFGS_B             : 200.00410991822957 	 N_evals : 4368      
	DA                   : 200.0000651705784 	 N_evals : 100003    
	L_BFGS               : 7.556706409884698e+17 	 N_evals : 1365      
	Scipy L_BFGS_B       : 283.9865821320329

We can that L-BFGS-B are generally more robust than similar algorithms implemented by Scipy and Minuit library.